In [ ]:
!pip install deepface opencv-python pandas matplotlib seaborn tqdm

In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from deepface import DeepFace
from tqdm import tqdm

25-02-13 20:49:01 - Directory /root/.deepface has been created
25-02-13 20:49:01 - Directory /root/.deepface/weights has been created


In [4]:
def process_video_emotions(video_path, output_plot="heatmap.png", frame_interval=1):

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Could not open video file")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    emotion_data = []
    frame_count = 0

    with tqdm(total=total_frames) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_interval == 0:
                # Convert frame to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                try:
                    # Analyze face and emotions
                    analysis = DeepFace.analyze(rgb_frame, actions=['emotion'], enforce_detection=True, silent=True)
                    if analysis:
                        emotions = analysis[0]['emotion']
                        emotions['frame'] = frame_count
                        emotions['time'] = frame_count / fps
                        emotion_data.append(emotions)
                except:
                    pass

            frame_count += 1
            pbar.update(1)

    cap.release()

    # Create DataFrame
    df = pd.DataFrame(emotion_data)
    if df.empty:
        raise ValueError("No faces detected in the video")

    # Normalize time for display
    df['time'] = df['time'].round(1)

    # Create heatmap
    plt.figure(figsize=(15, 8))
    heatmap_data = df.drop(['frame', 'time'], axis=1, errors='ignore')
    sns.heatmap(
        heatmap_data.T,
        xticklabels=df['time'],
        cmap='viridis',
        cbar_kws={'label': 'Emotion Intensity'}
    )

    plt.title("Emotion Heatmap Over Time")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Emotions")
    plt.yticks(rotation=0)
    plt.savefig(output_plot, bbox_inches='tight', dpi=300)
    plt.close()

    return df

# Example usage
if __name__ == "__main__":
    df = process_video_emotions(
        video_path="/content/input.mp4",
        output_plot="/content/emotion_heatmap.png",
        frame_interval=5  # Process every 5th frame for faster results
    )
    print("Processing complete. Heatmap saved as emotion_heatmap.png")

  0%|          | 0/1045 [00:00<?, ?it/s]

25-02-13 21:07:09 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 61.2MB/s]
100%|█████████▉| 1044/1045 [03:58<00:00,  4.37it/s]


Processing complete. Heatmap saved as emotion_heatmap.png
